# Web Information Retrieval
## Introduction to search engines

### DAY 2: Teacher version
### Implementing a search engine

The goal of this second session is to implement a first architecture of a search engine on the previously introduced dataset (stackexchange-datascience). If you missed the first session or if you did not saved the dataset, please reload the first session's notebook to download it. 

If you need some ifnormation about the dataset, it should be available here : https://archive.org/details/stackexchange

The notebook is divided into several steps:
-	Implement the indexation
-	Implement the search method
-	Define a ranking strategy and implement it
-	Suggest some improvements of the search engine



## Initialisation

In [1]:
import pandas as pd
import re
import os
import math
import numpy as np
from itertools import product
import numpy as np
import nltk 
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords

In [2]:
# TODO:
DATA_PATH = '/Users/hugov/EI_ST4_Groupe1/Data'


**Important :**

An Excel file for testing the evaluation part is available in the gitlab repo : evaluation_search_engine_post_queries_ranking_EI_CS.xlsx

If you work on Colab, we advice you to push it directly on your Google Drive directory.

# Implement the indexation
As you might already know, for a search engine to work properly an index of the documents must be created. Here we will keep it in python, and try to use only common libraries to keep it simple.

Once created, the index will be used to match the query with the documents. As a result, there are several ways to build an index, using statistical, boolean, semantic indexation...

First of, let's make a naive one that will consist in breaking down each document into a set of the words it contains.

In [3]:
import transformers
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')

def extract_words(text:str)->list:
  text2=text.lower()
  return tokenizer.tokenize(text2)

c:\Users\hugov\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
import pandas as pd

def index_docs(docs:list[str])->pd.DataFrame:
  data=[]
  for doc in docs:
    words=extract_words(doc)
    data.append([doc,words])

  return data

Now, let's try it on the dataset:

In [6]:
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")


,Id,PostTypeId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,...,ClosedDate,ContentLicense,AcceptedAnswerId,LastEditorUserId,LastEditDate,ParentId,OwnerDisplayName,CommunityOwnedDate,LastEditorDisplayName,FavoriteCount
0,5,1,2014-05-13T23:58:30.457,9,898.0,<p>I've always been interested in machine lear...,5.0,2014-05-14T00:36:31.077,How can I do simple machine learning without h...,<machine-learning>,...,2014-05-14T14:40:25.950,CC BY-SA 3.0,NaN,NaN,None,NaN,None,None,None,NaN
1,7,1,2014-05-14T00:11:06.457,4,478.0,"<p>As a researcher and instructor, I'm looking...",36.0,2014-05-16T13:45:00.237,What open-source books (or other materials) pr...,<education><open-source>,...,2014-05-14T08:40:54.950,CC BY-SA 3.0,10.0,97.0,2014-05-16T13:45:00.237,NaN,None,None,None,NaN
2,9,2,2014-05-14T00:36:31.077,5,NaN,"<p>Not sure if this fits the scope of this SE,...",51.0,2014-05-14T00:36:31.077,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,5.0,None,None,None,NaN
3,10,2,2014-05-14T00:53:43.273,13,NaN,"<p>One book that's freely available is ""The El...",22.0,2014-05-14T00:53:43.273,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,7.0,None,None,None,NaN
4,14,1,2014-05-14T01:25:59.677,26,1901.0,<p>I am sure data science as will be discussed...,66.0,2020-08-16T13:01:33.543,Is Data Science the Same as Data Mining?,<data-mining><definitions>,...,None,CC BY-SA 3.0,29.0,322.0,2014-06-17T16:17:20.473,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75722,119962,1,2023-03-04T20:06:06.820,0,8.0,<p>I am implementing a neural network of arbit...,147597.0,2023-03-04T20:22:12.523,Back Propagation on arbitrary depth network wi...,<neural-network><backpropagation>,...,None,CC BY-SA 4.0,NaN,147597.0,2023-03-04T20:22:12.523,NaN,None,None,None,NaN
75723,119963,1,2023-03-04T20:12:19.677,0,10.0,<p>I am using KNN for a regression task</p>\n<...,147598.0,2023-03-04T20:12:19.677,Evaluation parameter in knn,<regression><k-nn>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN
75724,119964,1,2023-03-05T00:14:12.597,0,7.0,<p>I have developed a small encoding algorithm...,44581.0,2023-03-05T00:14:12.597,Can I use zero-padded input and output layers ...,<deep-learning><convolutional-neural-network>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN
75725,119965,1,2023-03-05T00:43:12.213,0,5.0,"<p>To my understanding, optimizing a model wit...",84437.0,2023-03-05T00:43:12.213,Why does cross validation and hyperparameter t...,<cross-validation><hyperparameter-tuning>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN


For our first version of the indexation mechanism, we will simply use the "body" of the posts. To have a better search engine, the title and other metadata aswell could be used aswell. Finally, not all the XML files have a "body" feature, so for the search engine to retrieve information from any of the files you will need to implement another way to index.

But first, let's start with "body". There is more to preprocess than before, indeed, there are html tags such as "<p>" for instance. They are not useful for us, because users won't use them in their queries. So we first need to remove them.

In [7]:
import re

def remove_tags(text:str)->str:

  plain_text=re.sub('<.*?>', '', text)
  plain_text=re.sub('\n', ' ', plain_text)

  return plain_text

In [8]:
# test
remove_tags('<p>Hello World!\nI am making a search engine.<p>')

'Hello World! I am making a search engine.'

In [43]:
clean_posts = posts[['Id','Body']]
clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(remove_tags)
clean_posts

C:\Users\hugov\AppData\Local\Temp\ipykernel_9756\2440634092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(remove_tags)


,Id,Body,Clean Body
0,5,<p>I've always been interested in machine lear...,I've always been interested in machine learnin...
1,7,"<p>As a researcher and instructor, I'm looking...","As a researcher and instructor, I'm looking fo..."
2,9,"<p>Not sure if this fits the scope of this SE,...","Not sure if this fits the scope of this SE, bu..."
3,10,"<p>One book that's freely available is ""The El...","One book that's freely available is ""The Eleme..."
4,14,<p>I am sure data science as will be discussed...,I am sure data science as will be discussed in...
...,...,...,...
75722,119962,<p>I am implementing a neural network of arbit...,I am implementing a neural network of arbitrar...
75723,119963,<p>I am using KNN for a regression task</p>\n<...,I am using KNN for a regression task It's like...
75724,119964,<p>I have developed a small encoding algorithm...,I have developed a small encoding algorithm th...
75725,119965,"<p>To my understanding, optimizing a model wit...","To my understanding, optimizing a model with k..."


In [46]:
clean_posts['words'] = clean_posts['Clean Body'].apply(extract_words)

C:\Users\hugov\AppData\Local\Temp\ipykernel_9756\312806033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['words'] = clean_posts['Clean Body'].apply(extract_words)


In [52]:
clean_posts['length']=clean_posts['words'].apply(lambda x : len(x))
clean_posts
clean_posts['length'].mean()

C:\Users\hugov\AppData\Local\Temp\ipykernel_9756\694855702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['length']=clean_posts['words'].apply(lambda x : len(x))


260.10759702615974

## Inverted Index

Now, we want to go further on the indexing and build an inverted index. Inverted index is a dictionary where the keys are the words of the vocabulary and the values are the documents containing these words. Reducing the size of the vocabulary is a relevant first step when building an inverted index. Here, we will focus on the creation of the index, we leave you the optimisation steps :)

In [56]:


def create_index(posts:pd.DataFrame, number:int)-> dict :
  from nltk.corpus import stopwords
  stopwords=stopwords.words('english')
  extra=[',','?',':','!','.',';',"'",'"','-','(',')']
  stopwords.extend(extra)
  index={}
  for i in range(0,number):
    post=posts.iloc[i]
    Id=post['Id']
    for word in post['words']:
      if word not in stopwords:
        if word not in index:
          index[word]={Id : 1}
        else:
          if Id in index[word]:
            index[word][Id]+=1
          else:
            index[word][Id]=1
  return index
        

In [12]:

L=create_index(posts=clean_posts, number=5000)
L.keys()


dict_keys(['always', 'interested', 'machine', 'learning', 'figure', 'one', 'thing', 'starting', 'simple', 'hello', 'world', 'example', 'avoid', 'hard', 'coding', 'behavior', 'wanted', 'teach', 'b', '##ot', 'randomly', 'placed', 'obstacles', 'use', 'relative', 'motion', 'move', 'around', 'want', 'code', 'say', 'distance', 'ruins', 'whole', 'point', 'obviously', 'generating', 'would', 'imp', '##ractical', 'could', 'researcher', 'instructor', 'looking', 'open', 'source', 'books', '(', 'similar', 'materials', ')', 'provide', 'relatively', 'thorough', 'overview', 'data', 'science', 'applied', 'perspective', 'clear', 'especially', 'provides', 'material', 'suitable', 'college', 'level', 'course', 'particular', 'pieces', 'papers', 'sure', 'fits', 'scope', 'se', 'stab', 'answer', 'anyway', '##i', 'approaches', 'decide', 'modelling', 'kind', 'uncertainty', 'pick', 'framework', 'allows', 'situation', 'see', 'elements', 'fixed', 'flexible', 'model', 'may', 'allow', 'define', 'network', 'structure'

#### Well Done, you've indexed the dataset! 
Don't hesitate to save your indexes in txt or pickle file

---
# Implement the search method

A naive method would be to count the number of words in common between the query and each posts. Then to rank the posts you could directly select the post who maximize the number of common words. Let's implement this approach :

In [19]:

def rank_top_query(query, df, top=50):
  q=extract_words(query.lower())
  L={}
  index=create_index(posts=clean_posts, number = 50000)
  keys=index.keys()
  for word in q:
    if word in keys:
      for id in index[word].keys():
        if id in L:
          L[id]+=index[word][id]
        else:
          L[id]=index[word][id]
  sortedL=dict(sorted(L.items(), key=lambda item:item[1], reverse=True))
  print(sortedL)
  keys2=list(sortedL.keys())
  return keys2[:top]


In [54]:
rank_top_query(query="measure performance for multiclassification model", df=clean_posts, top=5)

{17621: 209, 69505: 164, 56860: 148, 23123: 116, 47875: 96, 61557: 92, 56196: 91, 65359: 82, 41798: 80, 62264: 72, 11189: 68, 38787: 68, 5078: 67, 13727: 67, 13701: 66, 5943: 65, 68922: 65, 9025: 64, 55294: 64, 40160: 62, 54796: 61, 62119: 61, 61302: 61, 42142: 60, 19160: 59, 69169: 59, 64790: 57, 20105: 56, 39483: 53, 61122: 53, 49350: 51, 18877: 50, 32292: 50, 42336: 49, 25227: 48, 36457: 48, 64100: 48, 64425: 48, 46740: 47, 60039: 46, 72314: 46, 71833: 45, 58649: 45, 33256: 45, 39396: 45, 23622: 44, 25207: 43, 56038: 43, 23998: 43, 16322: 42, 28992: 42, 38636: 42, 30990: 42, 51034: 42, 76905: 41, 36204: 41, 62316: 41, 63860: 41, 44612: 40, 47022: 40, 65074: 40, 38069: 40, 54626: 40, 43236: 39, 60379: 39, 71778: 39, 35887: 39, 37555: 39, 62202: 39, 43216: 39, 46127: 39, 67125: 38, 76634: 38, 27819: 38, 30385: 38, 9389: 37, 50825: 37, 55215: 37, 74293: 36, 24402: 36, 48117: 36, 33342: 36, 64743: 36, 68207: 36, 41903: 35, 36453: 35, 41224: 35, 25235: 34, 46824: 34, 68843: 34, 5932: 34,

[17621, 69505, 56860, 23123, 47875]

## Boolean Search

Thanks to the ttable library, implement a boolean search method

In [31]:
index=create_index(posts=clean_posts, number = 50000)
keys=index.keys()


def boolean_search_and(words:list):
  result=list(index[words[0]].keys())
  for word in words:
    if word in keys:
      result= list(set(result).intersection(index[word]))
    else:
      return False
  return result

  
  
boolean_search_and(['performance','data','science'])

[29699,
 9226,
 9227,
 48655,
 58383,
 24083,
 70163,
 23087,
 60472,
 14399,
 34383,
 55377,
 69201,
 47721,
 20084,
 629,
 26232,
 19586,
 49285,
 3719,
 31369,
 19085,
 65678,
 20113,
 49300,
 1173,
 9368,
 20129,
 12971,
 56496,
 68287,
 72390,
 14536,
 23243,
 19661,
 64719,
 17633,
 61153,
 12519,
 26869,
 12022,
 5368,
 65788,
 256,
 20230,
 779,
 71447,
 797,
 68896,
 26915,
 74041,
 39750,
 336,
 8017,
 339,
 5466,
 347,
 71517,
 24425,
 874,
 9587,
 49016,
 41848,
 37774,
 62357,
 51097,
 52126,
 415,
 60321,
 9635,
 43945,
 31658,
 73665,
 68556,
 29146,
 6131,
 28159]

## Probabilistic search

Implement the MIB or BM25 method of searching

In [60]:
#BM25


def probabilistic_search(query,    df,   k1=1.2,  b=0.75,  top=5):
  index=create_index(posts=df, number = 50000)
  size=len(df.index)
  query2=tokenizer.tokenize(query.lower())
  from nltk.corpus import stopwords
  stopwords=stopwords.words('english')
  extra=[',','?',':','!','.',';',"'",'"','-','(',')']
  stopwords.extend(extra)
  L={}
  query3=[]
  for word in query2:
    if word not in stopwords:
      query3.append(word)
  avg_D=df['length'].mean()
  for word in query3:
    DF=len(index[word])
    IDF=np.log(size-DF-0.5/DF+0.5)
    for doc in index[word]:
      D=len(df.loc[clean_posts['Id']==doc]['words'].tolist()[0])
      TF=index[word][doc]
      if doc in L:
        L[doc]+=IDF*TF*(k1+1)/((TF+k1)*(1-b+b*(D/avg_D)))
      else:
        L[doc]=IDF*TF*(k1+1)/((TF+k1)*(1-b+b*(D/avg_D)))
  
  
  sortedL=dict(sorted(L.items(), key=lambda item:item[1], reverse=True))
  keys2=list(sortedL.keys())
  return keys2[:top]

probabilistic_search(query='measure performance for multiclassification model', df=clean_posts)


[69657, 6619, 46296, 44666, 25179]

Compare the naive method with your improvements and the boolean and probabilistic search. (report)



---



---




# Evaluate the Search

Now you implement multiple search methods and you're able to improve it. You have to define metric to compare it objectively.



We ask you to implement NDCG (Normalized Discounted Cumulative Gain) from few queries we implement on a dozen of post. We already defined the values of relevance judgement in the xlsx file : . The final score will be the mean quadratic error of the queries.


Explication for the xlsx file :

We propose you a Excel file with some posts and a mesure of relevancy for the queries

- First column is the post Id,
- Columns starting by query are the queries you have to test.
- The values in this columns are the rank of relevancy of the post in regard with the query.
- The missing values indicates you should not take into account the post


You will have to criticize this metric and your result in the report. Then you will have to propose some improvements. 

Thereafter in this week, you will have to compare your different search engines.

In [ ]:
# Read Relevancy CSV
df_relevancy = pd.read_excel("/content/drive/MyDrive/TP Centrale/evaluation_search_engine_post_queries_ranking_EI_CS.xlsx")

In [ ]:
def calculate_ndgc(query_col="query", output_col="query_output"):
  # TODO

  return

